In [10]:
#@title
### EP2 MAC0417/MAC5768
##################################################################
# AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMEROUSP,  #
# DECLARO QUE SOU O ÚNICO AUTORE RESPONSÁVEL PELA RESOLUÇÃO      #
# DESTE EP.                                                      #
# TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM,   #
# SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM          #
# DESONESTIDADE ACADÊMICA OU PLÁGIO.                             #
#                                                                #
# DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS        #
# DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A          #
# SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E        #
# DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS    #
# DEFINIDOS NO CÓDIGO DE ÉTICA DA USP.                           #
#                                                                # 
# ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO         #
# CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR             #
# DESONESTIDADE ACADÊMICA.                                       #
#                                                                #
#                                                                #
# Nome: Luiz Gustavo Pina de Sales                               #
# NUSP: 10736991                                                 #
# Turma: 2024145                                                 #
# Prof.: Ronaldo Fumio Hashimoto                                 #
#################################################################

## Import Libraries

In [19]:
pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 1.2 MB/s eta 0:00:34
    --------------------------------------- 0.6/38.8 MB 4.8 MB/s eta 0:00:08
   - -------------------------------------- 1.0/38.8 MB 6.5 MB/s eta 0:00:06
   - -------------------------------------- 1.4/38.8 MB 6.5 MB/s eta 0:00:06
   - -------------------------------------- 1.9/38.8 MB 7.0 MB/s eta 0:00:06
   -- ------------------------------------- 2.3/38.8 MB 7.5 MB/s eta 0:00:05
   -- ------------------------------------- 2.8/38.8 MB 7.9 MB/s eta 0:00:05
   --- ------------------------------------ 3.3/38.8 MB 8.1 MB/s eta 0:00:05
   --- ------------------------------------ 3.9/38.8 MB 8.5 MB/s eta 0:00:05
   ---- ----------------------------------- 4.3/38.8 MB 8.7 MB/s eta 0:00:04
   ---- ----------------------------------- 4.8/38.8 MB 8.7 MB/s eta 0:00:04
   ----- ----

In [3]:
import cv2
from cv2 import resize, imwrite
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas
import os  

## Código principal para aquisição das imagens

No código abaixo temos um banco de imagens que irá conter um dicionário com a seguinte hierarquia de chaves:

    Classe -> Tempo -> Local -> Fundo -> Imagens

Todas as outras funções irão acessar as imagens através deste dicionário.

In [30]:
bancoImagens = {}
quantidadeClasses = 0
quantidadeImagens = 0
resolucaoImagens = ""

def loadImages(path, quantidadeClasses, quantidadeImagens, resolucaoImagens):
    imagens = os.listdir(path)
    
    for imagem in imagens:
        metadata = imagem.split(",")
        if metadata[0] not in bancoImagens.keys():
            bancoImagens[metadata[0]] = {}
            
        currentLevel = bancoImagens[metadata[0]]
        for metadataIndex in range(1,len(metadata)-1):
            if(metadata[metadataIndex] not in currentLevel): 
                currentLevel[metadata[metadataIndex]] = {}
                currentLevel = currentLevel[metadata[metadataIndex]]
            else: currentLevel = currentLevel[metadata[metadataIndex]] 

        local = metadata[-1].strip().replace(".jpg", "")
        currentLevel[local] = imread(path+imagem)
        quantidadeImagens += 1
        resolucaoImagens = f"{len(currentLevel[local])}x{len(currentLevel[local][0])}"
    return quantidadeClasses, quantidadeImagens, resolucaoImagens      
quantidadeClasses, quantidadeImagens, resolucaoImagens = loadImages("./Banco de Imagens/", quantidadeClasses, quantidadeImagens, resolucaoImagens)

## Tabela de Dados

In [35]:
def imprimeTabelaDetalhadaDeDados():
    print("\n=============================================================================\n")
    print("Tabela Detalhada de Dados:\n")
    dados = pandas.DataFrame({"Classe": [],
                         "N°Objetos": [], "N°Fundos": [],
                         "N°Iluminações": [], "N°repetições": [], 
                         "Total": []})
    classes = bancoImagens.keys()
    for classe in classes:
        totalAmostras = 0
        contadorRepeticoes = 0
        totalFundos = []
        totalTempos = []
        totalLocais = []
        tempos = bancoImagens[classe].keys()
        for tempo in tempos:
            if tempo not in totalTempos: totalTempos.append(tempo)
            for local in bancoImagens[classe][tempo].keys():
                if local not in totalLocais: totalLocais.append(local)
                for fundo in bancoImagens[classe][tempo][local].keys():
                    if fundo not in totalFundos: totalFundos.append(fundo)
                    contadorRepeticoes = len(bancoImagens[classe][tempo][local].keys())
                    
        contadorFundos = len(totalFundos)
        contadorIluminacoes = len(totalTempos)*len(totalLocais)
        totalAmostras += contadorIluminacoes*contadorFundos*contadorRepeticoes
                    
        dados = pandas.concat([dados, pandas.DataFrame({"Classe": classe, "N°Objetos": 1, "N°Fundos": contadorFundos,
        "N°Iluminações": contadorIluminacoes, "N°repetições": contadorRepeticoes,
         "Total": totalAmostras}, index=[0])], ignore_index = True)
    print(dados)
    print("\n===========================================================================\n")
imprimeTabelaDetalhadaDeDados()



Tabela Detalhada de Dados:

     Classe  N°Objetos  N°Fundos  N°Iluminações  N°repetições  Total
0   Alicate        1.0       6.0            4.0           3.0   72.0
1    Caneta        1.0       6.0            4.0           3.0   72.0
2    Colher        1.0       6.0            4.0           3.0   72.0
3  Controle        1.0       6.0            4.0           3.0   72.0
4   Esmalte        1.0       6.0            4.0           3.0   72.0
5      Faca        1.0       6.0            4.0           3.0   72.0
6     Garfo        1.0       6.0            4.0           3.0   72.0
7      Lixa        1.0       6.0            4.0           3.0   72.0
8     Moeda        1.0       6.0            4.0           3.0   72.0
9     Prato        1.0       6.0            4.0           3.0   72.0




## Save Image

In [37]:
def saveImage(filename, image):
    imwrite(filename, image)

## Funções de Equalização e Normalização de Histogramas

In [40]:
def plot_histogram(fig, titulo):
  plt.figure(figsize=(5,5))
  plt.plot(fig)
  plt.title(titulo)
  plt.show()

def normalized_histogram(image):
    normalizedValues = [0]*256
    for lineIndex in range(len(image)):
        for columnIndex in range(len(image[lineIndex])):
            normalizedValues[int(image[lineIndex][columnIndex])] +=1
            
    for i in range(len(normalizedValues)): 
        normalizedValues[i] /= (len(image)*len(image[0]))
    plot_histogram(normalizedValues, "Histograma normalizado")
    
    return normalizedValues
    
def histogram_equalization(originalImage, normalizedValues):
    image = originalImage.copy()
    plot_img(originalImage, "Before histogram equalization")
    for lineIndex in range(len(image)):
        for columnIndex in range(len(image[lineIndex])):
            normalizedSum = 0
            for value in range(int(image[lineIndex][columnIndex])): normalizedSum += normalizedValues[value]
            image[lineIndex][columnIndex] = 255*(normalizedSum)
    plot_img(image, "After histogram equalization")
    return image

## Histograma Médio dos Datasets

## Antes e Depois da Normalização

## Comentários